# Reddit Scraper

In [7]:
!pip install --quiet praw

In [128]:
import time
import datetime
import praw
import os
import traceback
import requests
import json
from tqdm.auto import tqdm
from pprint import pprint
import pandas as pd

In [9]:
b = "timestamp:"
d = ".."

Enter your config details here. For more info, visit https://github.com/reddit/reddit/wiki/OAuth2

In [10]:
from dotenv import load_dotenv
load_dotenv()
SECRET = os.getenv('REDDIT_APP_SECRET')
CLIENT_ID = os.getenv('REDDIT_APP_CLIENTID')
PASSOWRD = os.getenv('REDDIT_APP_PASSWORD')
AGENT = os.getenv('REDDIT_APP_AGENT')
USERNAME = os.getenv('REDDIT_APP_USERNAME')


In [11]:
r = praw.Reddit(client_id = CLIENT_ID,
                client_secret = SECRET,
                password = PASSOWRD,
                user_agent = AGENT,
                username = USERNAME)

## Checking if everything works alright

In [95]:
reddit = r
no_subreddit = reddit.subreddit('all')
hot = no_subreddit.hot(limit=3)
for i in hot:
    print(i)

ze013y
zdtwci
zdtaep


## Pun scraping from `r/puns`

In [174]:
SUBREDDIT = "puns"
LIMIT = 10000

In [175]:
subreddit2 = r.subreddit(SUBREDDIT)
#_subreddit = subreddit2.top(time_filter="all", limit=LIMIT)
r_subreddit = subreddit2.hot(limit=LIMIT)

In [176]:
post_type = set()
dict = { 
        "title":[],
        "body":[],
        "subreddit":[],
        "score":[], 
        "id":[], 
        "url":[], 
        "comms_num": [], 
        "created": [],
}

### Find post type
We only want text post

In [177]:
def get_post_type(subm):
    if getattr(subm, 'post_hint', '') == 'image':
        return 'image'
    elif getattr(subm, 'is_gallery', False):
        return 'gallery'
    elif subm.is_video:
        return 'video'
    elif hasattr(subm, 'poll_data'):
        return 'poll'
    elif hasattr(subm, 'crosspost_parent'):
        return "crosspost"
    elif subm.is_self:
        return 'text'
    return 'link'

### Driver code

In [178]:
for submission in tqdm(r_subreddit, total=LIMIT):
    cur_post_type = get_post_type(submission)
    post_type.add(cur_post_type)
    if cur_post_type == 'text':
        dict["title"].append(str(submission.title))
        dict["body"].append(str(submission.selftext))
        dict['subreddit'].append(str(submission.subreddit))
        dict["score"].append(str(submission.score))
        dict["id"].append(str(submission.id))
        dict["url"].append(str(submission.url))
        dict["created"].append(str(submission.created))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [170]:
print(f"Total text post scraped is { len(dict['title']) }")
df = pd.DataFrame.from_dict(dict).reset_index()
df

Total text post scraped is 364


,index,title,body,subreddit,score,id,url,comms_num,created
0,0,What did the apple(the company) said after not...,We will be MacInTouch,puns,2,ze39ad,https://www.reddit.com/r/puns/comments/ze39ad/...,1,1.670324e+09
1,1,A not so France-y experience part 2,"Not happy with the experience, the disappointe...",puns,0,ze2gp9,https://www.reddit.com/r/puns/comments/ze2gp9/...,0,1.670321e+09
2,2,A not so France-y experience,A tourist got robbed by some not-so-nice local...,puns,0,ze262j,https://www.reddit.com/r/puns/comments/ze262j/...,0,1.670320e+09
3,3,I mean......,If the air has gone somewhere.......\n\n\n\n.....,puns,0,ze1wxn,https://www.reddit.com/r/puns/comments/ze1wxn/...,0,1.670320e+09
4,4,I finally got revenge on the guy who wrote all...,"I ran at him, I pushed him down to the curb, a...",puns,2,zdvbpc,https://www.reddit.com/r/puns/comments/zdvbpc/...,1,1.670301e+09
...,...,...,...,...,...,...,...,...,...
359,359,Sponge Bob,A friend of mine is selling undersea real esta...,puns,0,xi4icf,https://www.reddit.com/r/puns/comments/xi4icf/...,1,1.663571e+09
360,360,Why don't Russians watch Netflix?,They have Njetflix,puns,9,xhih7r,https://www.reddit.com/r/puns/comments/xhih7r/...,4,1.663512e+09
361,361,My cat got arrested for vandalizing property.,Turns out they had probable claws …,puns,3,xhis9k,https://www.reddit.com/r/puns/comments/xhis9k/...,0,1.663513e+09
362,362,If you graduate cum laude,you'll probably make sapiosexuals cum laude.,puns,1,xhqfgo,https://www.reddit.com/r/puns/comments/xhqfgo/...,0,1.663531e+09


In [172]:
with open("scraped_joke.json", "w") as outfile:
    json.dump(dict, outfile)

TypeError: Object of type Subreddit is not JSON serializable